In [1]:
import folium
from folium.plugins import MarkerCluster, MousePosition
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import random
 
# Define the center of Dallas
dallas_center = [32.7767, -96.7970]
 
# Create 100 sample origins within Dallas
num_origins = 100
origins_data = {
    'Origin': [f'Origin_{i}' for i in range(num_origins)],
    'Latitude': [random.uniform(32.75, 32.80) for _ in range(num_origins)],
    'Longitude': [random.uniform(-96.83, -96.75) for _ in range(num_origins)]
}
origins = gpd.GeoDataFrame(origins_data, geometry=gpd.points_from_xy(origins_data['Longitude'], origins_data['Latitude']))
 
# Create 23 sample destinations within Dallas
num_destinations = 23
destinations_data = {
    'Destination': [f'Destination_{i}' for i in range(num_destinations)],
    'Latitude': [random.uniform(32.75, 32.80) for _ in range(num_destinations)],
    'Longitude': [random.uniform(-96.83, -96.75) for _ in range(num_destinations)]
}
destinations = gpd.GeoDataFrame(destinations_data, geometry=gpd.points_from_xy(destinations_data['Longitude'], destinations_data['Latitude']))
 
# Calculate travel times
travel_times = []
for _, origin_row in origins.iterrows():
    for _, dest_row in destinations.iterrows():
        # Generating random travel times (in minutes) between 10 and 60
        travel_time = random.uniform(10, 60)
        travel_times.append([origin_row['Origin'], dest_row['Destination'], travel_time])
 
# Create GeoDataFrame
df = gpd.GeoDataFrame(travel_times, columns=['Origin', 'Destination', 'TravelTime'])
 
# Create an interactive map centered around the mean latitude and longitude of all points
m = folium.Map(location=[(origins['Latitude'].mean() + destinations['Latitude'].mean()) / 2, 
                         (origins['Longitude'].mean() + destinations['Longitude'].mean()) / 2], 
               zoom_start=9)
 
# Add MousePosition plugin to show coordinates and travel time on hover
formatter = "function(num) {return L.Util.formatNum(num, 4) + ' º ';};"
folium.plugins.MousePosition(position='topright',
                              separator=' | ',
                              prefix="Coordinates:",
                              lat_formatter=formatter,
                              lng_formatter=formatter,
                              tooltip="Click to see coordinates").add_to(m)
 
# Dictionary to store LayerGroups for destination lines associated with each origin
destination_lines = {}
 
# Add marker clusters for destinations with a custom name
marker_cluster_destinations = MarkerCluster(name='Destinations').add_to(m)
 
# Add markers for destinations
for idx, destination in destinations.iterrows():
    folium.Marker([destination['Latitude'], destination['Longitude']], 
                  popup=f"Destination: {destination['Destination']}",
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(marker_cluster_destinations)
 
# Add lines for travel times
for idx, row in df.iterrows():
    origin_point = (origins[origins['Origin'] == row['Origin']].geometry.y.squeeze(),
                    origins[origins['Origin'] == row['Origin']].geometry.x.squeeze())
    dest_point = (destinations[destinations['Destination'] == row['Destination']].geometry.y.squeeze(),
                  destinations[destinations['Destination'] == row['Destination']].geometry.x.squeeze())
    # Create a PolyLine for each origin-destination pair
    travel_time_popup = f"Travel Time: {row['TravelTime']:.2f} units"  # Adjust units as needed
    line = folium.PolyLine([origin_point, dest_point], color="green", weight=2.5, opacity=1, popup=travel_time_popup)
    # Associate the PolyLine with its corresponding origin
    if row['Origin'] in destination_lines:
        destination_lines[row['Origin']].add_child(line)
    else:
        # Add the show=False argument to hide the layer by default
        destination_lines[row['Origin']] = folium.FeatureGroup(name=row['Origin'], show=False)
        destination_lines[row['Origin']].add_child(line)
        # Add the feature group to the map
        destination_lines[row['Origin']].add_to(m)
 
# Add markers for origins
for idx, origin in origins.iterrows():
    folium.Marker([origin['Latitude'], origin['Longitude']], 
                  popup=f"Origin: {origin['Origin']}",
                  icon=folium.Icon(color='red', icon='glyphicon-home')).add_to(destination_lines[origin['Origin']])
 
# Add LayerControl to toggle visibility of origin markers and destination lines
folium.LayerControl().add_to(m)
 
# Display the map
#m

# Import the required libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from ipywidgets import widgets, Layout, Box, VBox
from IPython.display import HTML

# Assume df, origins, and destinations are defined

# Create a dropdown widget with the same options as before
travel_time_dropdown = widgets.Dropdown(
    options=['All', '0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100'],
    value='All',
    description='Travel Time:',
    disabled=False,
)

# Create an output widget to display the map
map_output = widgets.Output()

# Define the update_map function as before
def update_map(change):
    # Clear the previous map output
    map_output.clear_output()
    
    # Get the selected option
    option = change['new']
    
    # Filter the travel times dataframe according to the option
    if option == 'All':
        filtered_df = df
    else:
        # Parse the lower and upper bounds of the range
        lower, upper = map(int, option.split('-'))
        # Filter the rows that fall within the range
        filtered_df = df[(df['TravelTime'] >= lower) & (df['TravelTime'] < upper)]
    
    # Create a new map object
    m = folium.Map(location=[(origins['Latitude'].mean() + destinations['Latitude'].mean()) / 2, 
                         (origins['Longitude'].mean() + destinations['Longitude'].mean()) / 2], 
               zoom_start=9)
    
    # Add the same plugins and markers as before
    folium.plugins.MousePosition(position='topright',
                              separator=' | ',
                              prefix="Coordinates:",
                              lat_formatter=formatter,
                              lng_formatter=formatter,
                              tooltip="Click to see coordinates").add_to(m)
    
    marker_cluster_destinations = MarkerCluster(name='Destinations').add_to(m)
    
    for idx, destination in destinations.iterrows():
        folium.Marker([destination['Latitude'], destination['Longitude']], 
                  popup=f"Destination: {destination['Destination']}",
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(marker_cluster_destinations)
    
    # Dictionary to store FeatureGroups for each origin
    origin_groups = {}
    
    # Add FeatureGroups for each origin
    for origin in origins['Origin'].unique():
        #origin_group = folium.FeatureGroup(name=origin).add_to(m)
        origin_group = folium.FeatureGroup(name=origin, show=False).add_to(m)
        origin_groups[origin] = origin_group
    
    # Add markers and lines for each origin-destination pair based on the filtered dataframe
    for idx, row in filtered_df.iterrows():
        origin_point = (origins[origins['Origin'] == row['Origin']].geometry.y.squeeze(),
                    origins[origins['Origin'] == row['Origin']].geometry.x.squeeze())
        dest_point = (destinations[destinations['Destination'] == row['Destination']].geometry.y.squeeze(),
                  destinations[destinations['Destination'] == row['Destination']].geometry.x.squeeze())
        travel_time_popup = f"Travel Time: {row['TravelTime']:.2f} units"
        # Add a marker for the origin
        origin_marker = folium.Marker(origin_point, 
                  popup=f"Origin: {row['Origin']}",
                  icon=folium.Icon(color='red', icon='map-marker'))
        # Add the marker to the corresponding origin group
        origin_groups[row['Origin']].add_child(origin_marker)
        # Add a line for the origin-destination pair
        line = folium.PolyLine([origin_point, dest_point], color="green", weight=2.5, opacity=1, popup=travel_time_popup)
        # Add the line to the corresponding origin group
        origin_groups[row['Origin']].add_child(line)
    
    # Add a LayerControl to the map
    folium.LayerControl().add_to(m)
    
    # Generate HTML code for the map
    map_html = m._repr_html_()
    
    # Display the map in the output widget
    with map_output:
        display(HTML(map_html))

# Call the update_map function with the initial value of the dropdown widget
update_map({'new': travel_time_dropdown.value})

# Link the dropdown widget to the update_map function
travel_time_dropdown.observe(update_map, names='value')

# Create a dashboard layout with the dropdown widget and the output widget
#dashboard = VBox([travel_time_dropdown, map_output])

# Display the dashboard using Voila
#dashboard

import pandas as pd
import folium
from folium.plugins import MarkerCluster
from ipywidgets import widgets, VBox
from IPython.display import display

# Assume df, origins, and destinations are defined

# Create a dropdown widget for selecting the origin
origin_dropdown = widgets.Dropdown(
    options=origins['Origin'].tolist(),
    description='Select Origin:',
    disabled=False,
)

# Create a dropdown widget for selecting the travel time
travel_time_dropdown = widgets.Dropdown(
    options=['All', '0-10', '10-20', '20-30', '30-40', '40-50', '50-60'],
    value='All',
    description='Travel Time:',
    disabled=False,
)

# Create an output widget to display the map
output = widgets.Output()

# Define the update_map function
def update_map(change):
    # Clear previous output
    output.clear_output()
    
    # Filter data based on selection
    selected_origin = origin_dropdown.value
    selected_time = travel_time_dropdown.value
    filtered_df = df[df['Origin'] == selected_origin]
    if selected_time != 'All':
        lower, upper = map(int, selected_time.split('-'))
        filtered_df = filtered_df[(filtered_df['TravelTime'] >= lower) & (filtered_df['TravelTime'] < upper)]
    
    # Create map centered around the mean latitude and longitude of all points
    m = folium.Map(location=[(origins['Latitude'].mean() + destinations['Latitude'].mean()) / 2, 
                              (origins['Longitude'].mean() + destinations['Longitude'].mean()) / 2], 
                   zoom_start=9)
    
    # Add marker for selected origin
    folium.Marker([origins.loc[origins['Origin'] == selected_origin, 'Latitude'].iloc[0],
                   origins.loc[origins['Origin'] == selected_origin, 'Longitude'].iloc[0]],
                  popup=f"Origin: {selected_origin}",
                  icon=folium.Icon(color='red', icon='glyphicon-home')).add_to(m)
    
    # Add markers for destinations in the selected travel time range
    marker_cluster_destinations = MarkerCluster(name='Destinations').add_to(m)
    for idx, row in filtered_df.iterrows():
        destination = row['Destination']
        dest_lat = destinations.loc[destinations['Destination'] == destination, 'Latitude'].iloc[0]
        dest_lon = destinations.loc[destinations['Destination'] == destination, 'Longitude'].iloc[0]
        folium.Marker([dest_lat, dest_lon], 
                      popup=f"Destination: {destination}",
                      icon=folium.Icon(color='blue', icon='map-marker')).add_to(marker_cluster_destinations)
    
    # Add markers for other origins as gray circles
    for idx, origin in origins.iterrows():
        if origin['Origin'] != selected_origin:
            folium.CircleMarker([origin['Latitude'], origin['Longitude']], 
                                radius=5, 
                                color='gray', 
                                fill=True, 
                                fill_color='gray', 
                                fill_opacity=0.6,
                                popup=f"Origin: {origin['Origin']}").add_to(m)
    
    # Display map
    with output:
        display(m)

# Call the update_map function with the initial value of the dropdown widgets
update_map(None)

# Link the dropdown widgets to the update_map function
origin_dropdown.observe(update_map, names='value')
travel_time_dropdown.observe(update_map, names='value')

# Create a dashboard layout with the dropdown widgets and the output widget
dashboard = VBox([origin_dropdown, travel_time_dropdown, output])

# Display the dashboard


In [2]:
#!pip install voila

In [3]:
display(dashboard)